# Clustering
In this series, we intend to provide implementations of the methods and algorithms described by [Rokach and Maimon 2005, ch 15](http://www.springer.com/us/book/9780387254654). Although these routines can be found on available packages like SciPy, the routines below are for educational purposes.

## Internal quality criteria
Internal quality metrics usually measure the compactness of the clusters using some similarity measure.

### Sum of Squared Errors
SSE is the simplest and most
widely used criterion measure for clustering. It is calculated as:
$$SSE = \sum_{k=1}^K \sum_{\forall x_i \in C_k} \|x_i-\mu_k\|^2$$
where $C_k$ is the set of instances in cluster k; $\mu_k$ is the vector mean of cluster
k. The components of $\mu_k$ are calculated as:
$$\mu_k=\frac{1}{N_k} \sum_{\forall x_i  \in C_k} x_{i,k}$$
Clustering methods that minimize the SSE criterion are often called minimum variance partitions, since by simple algebraic manipulation the SSE criterion may be written as:
$$SSE = \frac{1}{2} \sum_{k=1}^K N_k \bar S_k $$
Where:
$$\bar S_k=\frac{1}{n_k^2} \sum_{x_i,x_j \in C_k} \|x_i-x_j\|^2$$

First, we will establish a toy example to implement our quality criteria on. We will use the Iris dataset.

In [1]:
import numpy as np
from scipy.spatial import distance as spd
from sklearn import datasets
iris = datasets.load_iris()
dir(iris)

['DESCR', 'data', 'feature_names', 'target', 'target_names']

Then we implement standard $SSE$ calculation routine

In [2]:
def getLabs(data,target):
    # Listing unique labels
    labs = []
    for idx in range(len(target)):
        if not (target[idx] in labs):
            labs.append(target[idx])
    return labs

def getPosits(data,target):
    labs = getLabs(data,target)
    posits = []
    # Retrieving target positions
    for idy in range(len(labs)):
        temp=[]
        for idx in range(len(target)):
            if target[idx] == labs[idy]:
                temp.append(idx)
        posits.append(temp)
    return posits
    
def stdSSE(data,target):
    # Cluster means
    labs = getLabs(data,target)
    posits = getPosits(data,target)
    means = np.empty([len(labs),len(data[0])])
    for idy in range(len(labs)): means[idy] = np.mean(data[posits[idy]],axis=0)
    # SSE
    SSE=0
    for idy in range(len(labs)):
        for idx in posits[idy]:
            SSE = SSE + spd.euclidean(data[idx],means[idy])**2
    return SSE

In [3]:
stdSSE(iris.data,iris.target)

89.38680000000004

### Minimum Variance
We can now define the minimum variance method.

In [4]:
def minVar(data,target,dissimilarity):
    # Listing unique targets
    labs = getLabs(data,target)
    posits = getPosits(data,target)
    S = np.empty(len(labs))
    for idy in range(len(labs)):
        temp = 0
        for idxi in range(len(posits[idy])):
            for idxj in range(len(posits[idy])):
                temp = temp + dissimilarity(data[posits[idy][idxi]],data[posits[idy][idxj]])
        S[idy] = temp/(len(posits[idy])**2)
    SSE = 0
    for idy in range(len(labs)): SSE = SSE + float(len(posits[idy]))*S[idy]
    SSE = SSE/2
    return SSE
                

Them, for the euclidean distance (the classic form of SSE), one should provide the dataset and the squared euclidean function as argument. 

In [5]:
def quadEuclidean(a,b):
    res = spd.euclidean(a,b)**2
    return res
minVar(iris.data,iris.target,quadEuclidean)

89.386800000000022

The advantage of this construction is the possibility to use other similarity methods. A disadvantage is that while the direct construction is O(N), the minimum variance construction is of O(N^2).

In [19]:
print("manhattan minimum variance = ",minVar(iris.data,iris.target,spd.cityblock))
print("euclidean minimum variance = ", minVar(iris.data,iris.target,spd.euclidean))
print("Chebyshev minimum variance = ", minVar(iris.data,iris.target,spd.chebyshev))
print("Hamming minimum variance = ", minVar(iris.data,iris.target,spd.hamming))
print("Cosine minimum variance = ",minVar(iris.data,iris.target,spd.cosine))
print("Pearson minimum variance = ",minVar(iris.data,iris.target,spd.correlation))

manhattan minimum variance =  118.488
euclidean minimum variance =  70.370453723
Chebyshev minimum variance =  54.326
Hamming minimum variance =  65.4
Cosine minimum variance =  0.168183278276
Pearson minimum variance =  0.480793106828


### Scatter Criteria
The scatter criteria can be the within cluster criterion $S_B$, between cluster criterion $S_B$, total cluster criterion $S_T$ or a combination of them.
$S_W$ can be defined as:

In [7]:
def sWithin (data,target):
    labs = getLabs(data,target)
    posits = getPosits(data,target)
    Sk = np.zeros([len(data[0]),len(data[0])])
    for idy in range(len(labs)):
        cmean = np.mean(data[posits[idy]],axis=0)
        for vec in data[posits[idy]]:
            Sk = Sk + np.outer(vec-cmean,vec-cmean)
    return Sk

In [8]:
sWithin(iris.data,iris.target)

array([[ 38.9562,  13.683 ,  24.614 ,   5.6556],
       [ 13.683 ,  17.035 ,   8.12  ,   4.9132],
       [ 24.614 ,   8.12  ,  27.22  ,   6.2536],
       [  5.6556,   4.9132,   6.2536,   6.1756]])

And $S_B$:

In [9]:
def sBetween (data,target):
    labs = getLabs(data,target)
    posits = getPosits(data,target)
    Sb = np.zeros([len(data[0]),len(data[0])])
    cmean = 0
    tmean = np.zeros([len(data[0])])
    # Calculating the average of the cluster centers (which is equal to averaging over the whole dataset ¯\_(ツ)_/¯)
    for idy in range(len(labs)):
        tmean  = tmean + len(posits[idy])*np.mean(data[posits[idy]],axis=0)
    tmean = tmean/len(data)
    for idy in range(len(labs)):
        cmean = np.mean(data[posits[idy]],axis=0)
        Sb = Sb + float(len(posits[idy]))*np.outer(cmean-tmean,cmean-tmean)
    return Sb

In [10]:
sBetween(iris.data,iris.target)

array([[  63.21213333,  -19.534     ,  165.16466667,   71.36306667],
       [ -19.534     ,   10.9776    ,  -56.0552    ,  -22.4924    ],
       [ 165.16466667,  -56.0552    ,  436.64373333,  186.90813333],
       [  71.36306667,  -22.4924    ,  186.90813333,   80.60413333]])

And $S_T$:

In [11]:
def sTotal(data, target):
    tmean = np.mean(data,axis=0)
    St = np.zeros([len(data[0]),len(data[0])])
    for vec in data:
        St = St + np.outer(vec-tmean,vec-tmean)
    return St

In [12]:
sTotal(iris.data,iris.target)

array([[ 102.16833333,   -5.851     ,  189.77866667,   77.01866667],
       [  -5.851     ,   28.0126    ,  -47.9352    ,  -17.5792    ],
       [ 189.77866667,  -47.9352    ,  463.86373333,  193.16173333],
       [  77.01866667,  -17.5792    ,  193.16173333,   86.77973333]])

We can also generate scalars for the scatter criteria using:
* Trace criterion: calculating the sum of the diagonal of any matrix
* Determinant criterion: calculating the determinant of the matrix (does not apply if the number of clusters is less than the number dimensions)
* Invarian criterion: Based on the eigenvalues ($\lambda_i$) of the linear transformers (matrixes).
 1. $tr[S_{W}^{-1}S_B]=\sum\limits_{i=1}^d \lambda_i$
 2. $tr[S_{T}^{-1}S_W] = \sum\limits_{i=1}^d \frac{1}{1+\lambda_i}$
 3. $\frac{\mid S_w \mid}{\mid S_T \mid}=\prod\limits_{i=1}^{d} \frac{1}{1+\lambda_i}$ 

In [13]:
print("Trace Sw, Sb and St = ",np.trace(sWithin(iris.data,iris.target)),np.trace(sBetween(iris.data,iris.target)),\
     np.trace(sTotal(iris.data,iris.target)))
print("Determinant Sw, Sb and St = ",np.linalg.det(sWithin(iris.data,iris.target)),\
      np.linalg.det(sBetween(iris.data,iris.target)),\
      np.linalg.det(sTotal(iris.data,iris.target)))
print("Trace Sw^-1Sb = ",np.trace(np.matmul(np.linalg.inv(sWithin(iris.data,iris.target)),sBetween(iris.data,iris.target))))
print("Trace St^-1Sw = ",np.trace(np.matmul(np.linalg.inv(sTotal(iris.data,iris.target)),sWithin(iris.data,iris.target))))
print("|Sw|/|St| = ",np.linalg.norm(sWithin(iris.data,iris.target))/np.linalg.norm(sTotal(iris.data,iris.target)))

Trace Sw, Sb and St =  89.3868 591.4376 680.8244
Determinant Sw, Sb and St =  22069.1091687 5.5189899733e-27 938094.951014
Trace Sw^-1Sb =  32.5495246636
Trace St^-1Sw =  2.81279323666
|Sw|/|St| =  0.106313798265


### C-Criterion
We will skip the condorcet's criterion and go straight to the C-Criterion. The C-Criterion can be stated in terms of a similarity objective function to be maximized:
$$OF = \sum_{C_i \in C} \sum_{\substack{x_j,x_k \in C_i \\ x_j \neq X_k}}(s(x_j,x_k)-\gamma) + \sum_{C_i \in C} \sum_{\substack{x_j \in C \\ x_k \notin C_i}}(\gamma - s(x_j,x_k))$$
However, since we are dealing with numerical vectors, we will use a distance objective function to be minimzed:
$$OF = \sum_{C_i \in C} \sum_{\substack{x_j,x_k \in C_i \\ x_j \neq X_k}}(d(x_j,x_k)-\gamma) + \sum_{C_i \in C} \sum_{\substack{x_j \in C \\ x_k \notin C_i}}(\gamma - d(x_j,x_k))$$
Here, $\gamma$ is an arbitrary treshold.

In [14]:
def cCriterion(data,target,gamma=0,dissimilarity=spd.euclidean):
    from scipy.spatial import distance as spd
    labs = getLabs(data,target)
    posits = getPosits(data,target)
    s1 = 0
    s2 = 0
    # Calculating the first sum
    for idy in range(len(labs)):
        for idj in range(0,(len(posits[idy])-1)):
            for idk in range((idj+1),len(posits[idy])):
                s1 = s1 + dissimilarity(data[posits[idy][idj]],data[posits[idy][idk]])-gamma
    # calculating the second sum (yeah, four nested for loops, doesn't seem like an efficient metric)
    for idy in range((len(labs)-1)):
        for idz in range((idy+1),len(labs)):
            for idj in range(len(posits[idy])):
                for idk in range(len(posits[idz])):
                    s2 = s2 - gamma + dissimilarity(data[posits[idy][idj]],data[posits[idz][idk]])
    res = s1 + s2
    return res

28426.62094691259

In [21]:
cCriterion(iris.data,iris.target,gamma=0,dissimilarity=spd.euclidean)

28426.62094691259

## Forewords
We stop our quality criteria analisys here. We won't cover the subject of External Qualoty Criteria because this type of procedure will be better developed in the Supervised Learning notebooks.